In [1]:
import numpy as np
import pandas as pd
import os
import glob
import scipy 

In [2]:
path = '/Volumes/LaCie/school/combine_lab/Data/squamous_carcinoma/'

cSCC_counts.txt file is a merged raw count matrix of all patients (after 10x cellranger processing and filtered: cells with <200 genes or >10% mitochondiral gene expression were rm).

Thus, separate by patient and output the saved file:

In [ ]:
# cSCC_counts = pd.read_csv(path+'scRNAseq/'+'GSE144236_cSCC_counts.txt', sep = '\t', chunksize=1000)

# # merge all chunks into one df
# scRNAseq = pd.concat([chunk for chunk in cSCC_counts])

# # split df by patient
# cSCC_scRNA = []
# for i, j in scRNAseq.groupby(scRNAseq.columns.str.split("_").str[0], axis=1):
#     cSCC_scRNA.append(j)
    
# # save df by patient idex to separate CSV files
# for i, j in scRNAseq.groupby(scRNAseq.columns.str.split("_").str[0], axis=1):
#     j.to_csv('Data/squamous_carcinoma/scRNAseq_{}.csv'.format(i), header=True, index_label=False)

In [3]:
# squamous carcinoma dataset
path = '/Volumes/LaCie/school/combine_lab/Data/squamous_carcinoma/'

all_counts = pd.read_csv(path+'ST/'+'GSE144239_ST_all_counts.txt', sep = '\t')
st_barcodes = pd.read_csv(path+'ST/'+'GSE144239_ST_barcode_id.txt', sep = '\t', header=None)
visium = pd.read_csv(path+'ST/'+'GSE144239_ST_Visium_counts.txt', sep = '\t')

metadata = pd.read_csv(path+'scRNAseq/'+'GSE144236_patient_metadata_new.txt', sep = '\t')

In [4]:
# read in all patient scRNAseq files
scRNA = [pd.read_csv(j, index_col=0) for j in glob.glob(os.path.join(path, '*.csv'))]

In [5]:
range(len(scRNA)) # scRNA for 10 patients

range(0, 10)

In [6]:
scRNA[0]

,P1_Tumor_AAACCTGAGTCAAGCG,P1_Tumor_AAACCTGCAAATTGCC,P1_Tumor_AAACCTGGTAGGAGTC,P1_Tumor_AAACGGGAGATGTAAC,P1_Tumor_AAACGGGAGCTGCAAG,P1_Tumor_AAAGATGAGAAGGTTT,P1_Tumor_AAAGATGCAATAACGA,P1_Tumor_AAAGATGCAGACTCGC,P1_Tumor_AAAGATGCAGTCAGCC,P1_Tumor_AAAGATGGTGGTGTAG,...,P1_Normal_TTGGCAAGTGTTCTTT,P1_Normal_TTGGCAATCCTGCTTG,P1_Normal_TTGTAGGAGGTGCTTT,P1_Normal_TTGTAGGCAAGAAGAG,P1_Normal_TTGTAGGTCCTGCTTG,P1_Normal_TTTACTGGTCTACCTC,P1_Normal_TTTATGCGTTTGTTGG,P1_Normal_TTTCCTCAGGAGCGAG,P1_Normal_TTTCCTCCACGGCGTT,P1_Normal_TTTCCTCTCACAGTAC
Patient,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
"Tissue: 0=Normal, 1=Tumor",1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
RP11-34P13.7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AL627309.1,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
AP006222.2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AC145205.1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
BAGE5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CU459201.1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AC002321.2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
all_counts

,P2_7x45_1,P2_8x30_1,P2_8x32_1,P2_8x34_1,P2_8x36_1,P2_8x38_1,P2_8x44_1,P2_8x46_1,P2_9x29_1,P2_9x31_1,...,P5_41x27_3,P5_41x31_3,P5_41x33_3,P5_41x35_3,P5_41x37_3,P5_41x39_3,P5_41x41_3,P5_42x36_3,P5_42x38_3,P5_42x40_3
Patient,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0
Section replicate,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
pixel_x,3282.6,3473.4,3471.9,3474.1,3469.5,3467.4,3471.4,3462.3,3650.9,3649.9,...,9448.4,9446.6,9437.3,9451.4,9451.1,9445.3,9451.1,9634.2,9628.1,9623.4
pixel_y,9536.7,6539.5,6937.9,7338.8,7736.4,8137.2,9337.8,9738.9,6336.8,6737.0,...,6984.0,7809.0,8205.3,8613.4,9011.1,9412.5,9810.7,8815.8,9224.7,9620.7
FO538757.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MAGEA2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
MAGEA6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
LCE5A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
NPIPA8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


ST for P2, 5, 9 and 10 was generated with an older version of technology with larget spot size and center-to-center distance.

In [8]:
st_keep = np.unique(all_counts.loc['Patient'].values)
print(st_keep)
# type(st_keep)
st_keep = st_keep.tolist()
st_keep = [int(i) for i in st_keep] # turn float to int
print(st_keep)

[ 2.  5.  9. 10.]
[2, 5, 9, 10]


In [ ]:
np.unique(all_counts.loc['Section replicate'].values)

In [ ]:
print(type(scRNA))
print(type(scRNA[0]))

Keep only scRNA for patients 2, 5, 9, and 10

In [9]:
scrna = []
for i in range(len(scRNA)):
    if scRNA[i].loc['Patient'].isin(st_keep).any():
        scrna.append(scRNA[i])
len(scrna)

4

In [ ]:
scrna[3]

In [10]:
counts = []
for i in range(len(scrna)):
    counts.append(scrna[i].drop(['Patient', 'Tissue: 0=Normal, 1=Tumor'], axis=0)) # delete first two rows

In [15]:
counts[0]

,P10_Tumor_AAACCTGGTCATGCCG,P10_Tumor_AAACCTGTCGACAGCC,P10_Tumor_AAACCTGTCGCTGATA,P10_Tumor_AAACGGGCACTTACGA,P10_Tumor_AAACGGGTCACTTATC,P10_Tumor_AAAGCAAAGACTAGAT,P10_Tumor_AAAGCAAAGAGCTATA,P10_Tumor_AAAGCAAAGATTACCC,P10_Tumor_AAAGCAACAGCTGGCT,P10_Tumor_AAAGCAACATCCCATC,...,P10_Normal_TTTGGTTCACTCTGTC,P10_Normal_TTTGGTTCATCATCCC,P10_Normal_TTTGGTTCATCCCATC,P10_Normal_TTTGGTTTCTCTAAGG,P10_Normal_TTTGTCAAGACAATAC,P10_Normal_TTTGTCAAGAGTCTGG,P10_Normal_TTTGTCAAGTCGTTTG,P10_Normal_TTTGTCACAGTGGGAT,P10_Normal_TTTGTCATCCAGGGCT,P10_Normal_TTTGTCATCGAGGTAG
RP11-34P13.7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AL627309.1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AP006222.2,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
RP4-669L17.10,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
RP11-206L10.3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AC145205.1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
BAGE5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CU459201.1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AC002321.2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [36]:
from scipy.sparse import csr_matrix

def pd2csr(df):
    #from dense to sparse, use df.astype() with a SparseDtype
    spars_dtype = pd.SparseDtype(int, fill_value=0)
    spars_df = df.astype(spars_dtype)

    # convert to coo()
    df_coo = spars_df.sparse.to_coo()

    # convert to col sparse row matrix
    return csr_matrix((df_coo.data, (df_coo.row, df_coo.col)))

In [34]:
scipy.sparse.save_npz('Data/squamous_carcinoma/')

<32736x4569 sparse matrix of type '<class 'numpy.int64'>'
	with 13558936 stored elements in Compressed Sparse Row format>

In [37]:
spar_count = []
for i in range(len(counts)):
    spar_count.append(pd2csr(counts[i]))

In [43]:
for i in range(len(spar_count)):
    scipy.sparse.save_npz(f"Data/squamous_carcinoma/scRNA_counts_{i}.npz", spar_count[i])

10X Visium used for P4 and 6 (not at sc but better resolution than older version).

Note, spatial spot barcodes are also 16-oligo sequence.

In [ ]:
visium

In [ ]:
st_barcodes

In [ ]:
# split spot barcode on _ and keep just the ending
spt = []
for i in visium.columns:
    head, sep, tail = i.partition('_')
    spt.append(tail)

# split spot barcode on - and keep just first part
spot = []
for i in spt:
    spot.append(i.split('-', 1)[0])
np.unique(spot)

In [ ]:
# count number of unique spots
len(np.unique(spot))

In [ ]:
spots = np.unique(spot).astype(object)
spots

In [ ]:
st_barcodes.loc[:, 0]

In [ ]:
st_barcodes.loc[:, 0].isin(spots)

In [ ]:
st_barcodes[st_barcodes.loc[:, 0].isin(spots)]

In [ ]:
for i in range(len(spot)):
    if spot[i] in st_barcodes[0] == True:
        print(spot[i])
#     # else:
#     #     print('no spot in barcode')

In [ ]:
for i in range(len(st_barcodes[0])):
    if st_barcodes[0][i] in spot == True:
        print(st_barcodes[0][i])
    # else:
    #     print("barcode not in spot")

In [ ]:
print(np.unique(visium.iloc[0].values)) # find unique patients 
print(np.unique(visium.iloc[1].values)) # find unique replicates 

In [ ]:
metadata

In [ ]:
# count number of unique cells
barcds = [x.split('_')[2] for x in metadata.index] # split rownames on '_' and keep barcode only
np.unique(barcds).shape 

In [ ]:
metadata['patient'].unique()

In [ ]:
len(scRNA) # number of patients = 10 (as expected)